In [1]:
import os
import re
from dataclasses import dataclass
from reactionmodel.model import Species, Reaction, ReactionRateFamily, Model

In [2]:
a = Species('A')
r = Reaction([a], [])
m = Model([a], [r])

In [3]:
m.all_reactions

[Reaction(description=, reactants=(Species(name='A', description=''),), products=(), kinetic_order=(Species(name='A', description=''),), k=None)]

In [4]:
m.get_k(reaction_to_k={r:1})

array([1.])

In [5]:
r.to_dict()

{'reactants': ({'name': 'A', 'description': ''},),
 'products': (),
 'description': '',
 'rate_involved': ({'name': 'A', 'description': ''},),
 'reversible': False,
 'k': None}

In [6]:
m.to_dict()

{'species': [{'name': 'A', 'description': ''}],
 'reactions': [{'reactants': ({'name': 'A', 'description': ''},),
   'products': (),
   'description': '',
   'rate_involved': ({'name': 'A', 'description': ''},),
   'reversible': False,
   'k': None}]}

In [7]:
m2 = m.from_dict(m.to_dict())

[Reaction(description=, reactants=(Species(name='A', description=''),), products=(), kinetic_order=(Species(name='A', description=''),), k=None)]


In [8]:
a = Species('A')
r1 = Reaction([a], [])
r2 = Reaction([(a, 2)], [])
r_g = ReactionRateFamily([r1, r2], k='hi')

m = Model([a], [r1,r2])
m.to_dict()

{'species': [{'name': 'A', 'description': ''}],
 'reactions': [{'reactants': ({'name': 'A', 'description': ''},),
   'products': (),
   'description': '',
   'rate_involved': ({'name': 'A', 'description': ''},),
   'reversible': False,
   'k': None},
  {'reactants': (({'name': 'A', 'description': ''}, 2),),
   'products': (),
   'description': '',
   'rate_involved': (({'name': 'A', 'description': ''}, 2),),
   'reversible': False,
   'k': None}]}

In [9]:
import yaml
print(yaml.dump(m.to_dict()))

reactions:
- description: ''
  k: null
  products: !!python/tuple []
  rate_involved: !!python/tuple
  - description: ''
    name: A
  reactants: !!python/tuple
  - description: ''
    name: A
  reversible: false
- description: ''
  k: null
  products: !!python/tuple []
  rate_involved: !!python/tuple
  - !!python/tuple
    - description: ''
      name: A
    - 2
  reactants: !!python/tuple
  - !!python/tuple
    - description: ''
      name: A
    - 2
  reversible: false
species:
- description: ''
  name: A



In [10]:
@dataclass(frozen=True)
class Syntax():
    family_denoter = '$'
    family_enumerator = '#'

from itertools import product
def enumerated_product(members):
    yield from zip(product(*(range(len(x)) for x in members)), product(*members))

def family_replace(family_names, idx, chosen_members, value, syntax=Syntax(), do_nestings=False):
    for family_name, member, i in zip(family_names, chosen_members, idx):
        if do_nestings and isinstance(value, (tuple, list)):
            value = [family_replace(family_names, idx, chosen_members, v, syntax, do_nestings) for v in value]
            continue
        if do_nestings and isinstance(value, dict):
            value = {k:family_replace(family_names, idx, chosen_members, v, syntax, do_nestings) for k,v in value.items()}
            continue
        if isinstance(value, (int, float)):
            continue
        value = value.replace(syntax.family_denoter + family_name, member)
        value = value.replace(syntax.family_enumerator + family_name, str(i))
    return value

from enum import Enum
class ReactionRateFamilyApplicationMethod(Enum):
    group = 'group'
    split = 'split'

def expand_families(families, atom, syntax=Syntax()):
    try:
        used_families = atom.pop('used_families')
    except KeyError:
        return [atom]
    nested_fields = ['products', 'reactants']
    # are we a ReactionRateFamily?
    if 'reactions' in atom.keys():
        # we are a ReactionRateFamily
        try:
            family_method = ReactionRateFamilyApplicationMethod(atom.pop('family_method'))
        except KeyError:
            family_method = ReactionRateFamilyApplicationMethod.group
        
        if family_method == ReactionRateFamilyApplicationMethod.group:
            for field, value in atom.items():
                if field != 'reactions':
                    assert syntax.family_denoter not in value
                    assert syntax.family_enumerator not in value
                    continue

            new_reactions = []
            for r in atom['reactions']:
                r_ = r.copy()
                r_['used_families'] = used_families
                new = expand_families(families, r_, syntax)
                print(new)
                new_reactions.extend(new)
            new_rrf = atom.copy()
            new_rrf['reactions'] = new_reactions
            return new_rrf
        elif family_method == ReactionRateFamilyApplicationMethod.split:
            # we want to continue with expansion, but make a note that reactions will have to be expanded as well
            nested_fields.append('reactions')
    family_members = []
    for _, global_family_name in used_families.items():
        members = families[global_family_name]
        # append the *list*, so we can keep our families straight
        family_members.append(members)

    new_atoms = []
    for idx, combination in enumerated_product(family_members):
        new_atom = atom.copy()
        for field, value in atom.items():
            new_atom[field] = family_replace(used_families, idx, combination, value, syntax=syntax, do_nestings=(field in nested_fields))
        new_atoms.append(new_atom)
    
    return new_atoms

def parse_model(families, species, reactions):
    all_species = []
    for s in species:
        all_species.extend(expand_families(families, s))
    all_reactions = []
    for r in reactions:
        all_reactions.extend(expand_families(families, r))
    model_dict = {
        'species'  : all_species,
        'reactions': all_reactions,
    }
    print(model_dict)
    return Model.from_dict(model_dict)

In [11]:
expand_families({'fam':['foo', 'bar']}, {'x':'$i', 'y':'$j#i', 'used_families':{'i':'fam', 'j':'fam'}})

[{'x': 'foo', 'y': 'foo0'},
 {'x': 'foo', 'y': 'bar0'},
 {'x': 'bar', 'y': 'foo1'},
 {'x': 'bar', 'y': 'bar1'}]

In [12]:
expand_families({'fam':['foo', 'bar']}, {'x':'$i', 'y':'$j#i', 'reactions':[{'a':'$i'}, {'a':'$j'}], 'used_families':{'i':'fam', 'j':'fam'}, 'family_method':'split'})

[{'x': 'foo', 'y': 'foo0', 'reactions': [{'a': 'foo'}, {'a': 'foo'}]},
 {'x': 'foo', 'y': 'bar0', 'reactions': [{'a': 'foo'}, {'a': 'bar'}]},
 {'x': 'bar', 'y': 'foo1', 'reactions': [{'a': 'bar'}, {'a': 'foo'}]},
 {'x': 'bar', 'y': 'bar1', 'reactions': [{'a': 'bar'}, {'a': 'bar'}]}]

In [13]:
expand_families({'fam':['foo', 'bar']}, {'reactions':[{'a':'$i'}, {'a':'$j'}], 'used_families':{'i':'fam', 'j':'fam'}, 'family_method':'group'})

[{'a': 'foo'}, {'a': 'foo'}, {'a': 'bar'}, {'a': 'bar'}]
[{'a': 'foo'}, {'a': 'bar'}, {'a': 'foo'}, {'a': 'bar'}]


{'reactions': [{'a': 'foo'},
  {'a': 'foo'},
  {'a': 'bar'},
  {'a': 'bar'},
  {'a': 'foo'},
  {'a': 'bar'},
  {'a': 'foo'},
  {'a': 'bar'}]}

In [14]:
from yaml import SafeLoader as Loader

yaml_string = """
families:
  As: [x, y, z]
  Bs: [p, q, r]

species:
- name: 'A_$i'
  used_families: {'i': 'As'}
- name: 'B_$j'
  used_families: {'j': 'Bs'}

reactions:
- reactants: [['A_$i',2]]
  products: ['B_$j']
  k: 'Q[#i][#j]'
  used_families: {'i': 'As', 'j': 'Bs'}
"""

md = yaml.load(yaml_string, Loader=Loader)
md

{'families': {'As': ['x', 'y', 'z'], 'Bs': ['p', 'q', 'r']},
 'species': [{'name': 'A_$i', 'used_families': {'i': 'As'}},
  {'name': 'B_$j', 'used_families': {'j': 'Bs'}}],
 'reactions': [{'reactants': [['A_$i', 2]],
   'products': ['B_$j'],
   'k': 'Q[#i][#j]',
   'used_families': {'i': 'As', 'j': 'Bs'}}]}

In [15]:
m = parse_model(md['families'], md['species'], md['reactions'])

{'species': [{'name': 'A_x'}, {'name': 'A_y'}, {'name': 'A_z'}, {'name': 'B_p'}, {'name': 'B_q'}, {'name': 'B_r'}], 'reactions': [{'reactants': [['A_x', 2]], 'products': ['B_p'], 'k': 'Q[0][0]'}, {'reactants': [['A_x', 2]], 'products': ['B_q'], 'k': 'Q[0][1]'}, {'reactants': [['A_x', 2]], 'products': ['B_r'], 'k': 'Q[0][2]'}, {'reactants': [['A_y', 2]], 'products': ['B_p'], 'k': 'Q[1][0]'}, {'reactants': [['A_y', 2]], 'products': ['B_q'], 'k': 'Q[1][1]'}, {'reactants': [['A_y', 2]], 'products': ['B_r'], 'k': 'Q[1][2]'}, {'reactants': [['A_z', 2]], 'products': ['B_p'], 'k': 'Q[2][0]'}, {'reactants': [['A_z', 2]], 'products': ['B_q'], 'k': 'Q[2][1]'}, {'reactants': [['A_z', 2]], 'products': ['B_r'], 'k': 'Q[2][2]'}]}
[Reaction(description=, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=(), k=Q[0][0]), Reaction(description=, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_q', descripti

In [16]:
m.all_reactions

[Reaction(description=, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=(), k=Q[0][0]),
 Reaction(description=, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=(), k=Q[0][1]),
 Reaction(description=, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_r', description=''),), kinetic_order=(), k=Q[0][2]),
 Reaction(description=, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=(), k=Q[1][0]),
 Reaction(description=, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=(), k=Q[1][1]),
 Reaction(description=, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_r', description=''),), kinetic_order=(), k=Q[1][2]),
 Reaction(description=, reactants=((Species(name='A_

In [17]:
def localize_string_with_family_members(string, syntax, families, chosen_members, idx):
    for family_name, member, i in zip(families, chosen_members, idx):
        string = string.replace(syntax.family_denoter + family_name, member)
        string = string.replace(syntax.family_enumerator + family_name, str(i))
    return string

def add_atoms(self, existing_atoms, factory, atom_name, atom_properties):
    new_atoms = {}
    if self.syntax.family_denoter in atom_name:
        #import pdb; pdb.set_trace()
        families = set(re.findall(self.family_pattern, atom_name))
        families = re.findall(self.family_pattern, atom_name)

        used_families_property_name = None
        for p in factory.properties:
            if isinstance(p, UsedFamiliesProperty):
                used_families_property_name = p.name
        if used_families_property_name is None:
            raise ParserSyntaxError(f"atom {atom_name} used the family token {self.syntax.family_denoter} but that atom type doesn't support families.")

        used_families_property = atom_properties.get(used_families_property_name, None)
        if used_families_property is None:
            raise ParserSyntaxError(f"atom {atom_name} used the family token {self.syntax.family_denoter} but no {used_families_property_name} property was specified.")

        # teach the used families property about the families that exist in our specification
        used_families_property = used_families_property.evaluate_with_existing_atoms(existing_atoms)

        family_members = []
        for f in families:
            family = used_families_property.get(f, None)
            if family is None or not(isinstance(family, Family)):
                raise ParserSyntaxError(f"looked for family {f} but couldn't find its alias in {used_families_property_name}.")
            # append the *list*, so we can keep our families straight
            family_members.append(family.members)
        for idx, combination in self.enumerated_product(family_members):
            localized_name = PropertyMatch.localize_string_with_family_members(atom_name, self.syntax, families, combination, idx)
            localized_properties = self.localize_properties_with_family_members(atom_properties, families, combination, idx)
            new_atoms[localized_name] = self.construct_atom(existing_atoms, factory, localized_name, localized_properties)
    else:
        new_atoms[atom_name] = self.construct_atom(existing_atoms, factory, atom_name, atom_properties)
    #import pdb; pdb.set_trace()
    existing_atoms.update(new_atoms)
    return existing_atoms

In [18]:
import reactionmodel.smallparser as smallparser

In [19]:
from yaml import SafeLoader as Loader

yaml_string = """
families:
  As: [x, y, z]
  Bs: [p, q, r]

species:
- name: 'A_$i'
  used_families: {'i': 'As'}
- name: 'B_$j'
  used_families: {'j': 'Bs'}

reactions:
- description: 2A_$i => B_$j
  reactants: [['A_$i',2]]
  products: ['B_$j']
  k: 'Q[#i][#j]'
  used_families: {'i': 'As', 'j': 'Bs'}

parameters:
 foo: 2.0
 bar: {'path': './examples/minimal/q.csv'}

simulator_config:
  simulator: gillespie

initial_condition:
  A_$i: 1.0 * foo
  used_families: {'i': 'As'}
"""

md = yaml.load(yaml_string, Loader=Loader)
md

{'families': {'As': ['x', 'y', 'z'], 'Bs': ['p', 'q', 'r']},
 'species': [{'name': 'A_$i', 'used_families': {'i': 'As'}},
  {'name': 'B_$j', 'used_families': {'j': 'Bs'}}],
 'reactions': [{'description': '2A_$i => B_$j',
   'reactants': [['A_$i', 2]],
   'products': ['B_$j'],
   'k': 'Q[#i][#j]',
   'used_families': {'i': 'As', 'j': 'Bs'}}],
 'parameters': {'foo': 2.0, 'bar': {'path': './examples/minimal/q.csv'}},
 'simulator_config': {'simulator': 'gillespie'},
 'initial_condition': {'A_$i': '1.0 * foo', 'used_families': {'i': 'As'}}}

In [20]:
p = smallparser.loads(md)

{'species': [{'name': 'A_x'}, {'name': 'A_y'}, {'name': 'A_z'}, {'name': 'B_p'}, {'name': 'B_q'}, {'name': 'B_r'}], 'reactions': [{'description': '2A_x => B_p', 'reactants': [['A_x', 2]], 'products': ['B_p'], 'k': 'Q[0][0]'}, {'description': '2A_x => B_q', 'reactants': [['A_x', 2]], 'products': ['B_q'], 'k': 'Q[0][1]'}, {'description': '2A_x => B_r', 'reactants': [['A_x', 2]], 'products': ['B_r'], 'k': 'Q[0][2]'}, {'description': '2A_y => B_p', 'reactants': [['A_y', 2]], 'products': ['B_p'], 'k': 'Q[1][0]'}, {'description': '2A_y => B_q', 'reactants': [['A_y', 2]], 'products': ['B_q'], 'k': 'Q[1][1]'}, {'description': '2A_y => B_r', 'reactants': [['A_y', 2]], 'products': ['B_r'], 'k': 'Q[1][2]'}, {'description': '2A_z => B_p', 'reactants': [['A_z', 2]], 'products': ['B_p'], 'k': 'Q[2][0]'}, {'description': '2A_z => B_q', 'reactants': [['A_z', 2]], 'products': ['B_q'], 'k': 'Q[2][1]'}, {'description': '2A_z => B_r', 'reactants': [['A_z', 2]], 'products': ['B_r'], 'k': 'Q[2][2]'}]}
[Reac

In [21]:
p.initial_condition

{'A_x': '1.0 * foo', 'A_y': '1.0 * foo', 'A_z': '1.0 * foo'}

In [22]:
p.model.make_initial_condition(p.initial_condition, p.parameters)

Evaluating expression: 1.0 * foo => 2.0
Evaluating expression: 1.0 * foo => 2.0
Evaluating expression: 1.0 * foo => 2.0


array([2., 2., 2., 0., 0., 0.])

In [23]:
p.parameters

{'foo': 2.0,
 'bar': array([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])}

In [24]:
p.simulator_config

{'simulator': 'gillespie'}

In [25]:
p.model.all_reactions

[Reaction(description=2A_x => B_p, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=(), k=Q[0][0]),
 Reaction(description=2A_x => B_q, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=(), k=Q[0][1]),
 Reaction(description=2A_x => B_r, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_r', description=''),), kinetic_order=(), k=Q[0][2]),
 Reaction(description=2A_y => B_p, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=(), k=Q[1][0]),
 Reaction(description=2A_y => B_q, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=(), k=Q[1][1]),
 Reaction(description=2A_y => B_r, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_r', description=''),), kinetic_order=()